In [1]:
# import library

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from PIL import Image
from datetime import date


In [2]:
# Save keys

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
import csv
from datetime import date
def log_anatomy_to_csv(anatomy, filename, folder="anatomy_posters", csv_path="anatomy_poster_log.csv"):
    today = date.today().isoformat()
    filepath = os.path.join(folder, filename)

    row = {
        "anatomy": anatomy,
        "filename": filename,
        "date": today,
        "path": filepath
    }

    file_exists = os.path.isfile(csv_path)

    with open(csv_path, mode='a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["anatomy", "filename", "date", "path"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()

        writer.writerow(row)

    print(f"📝 CSV updated: {csv_path}")



In [4]:
## List of anatomy medical
medical_anatomy = [
    "Heart", "Lung", "Kidney", "Eye", "Brain", "Bladder", "Abdomen", "Uterus",
    "Skeleton", "Muscles", "Nervous system", "Venous system", "Arterial system", "Ear", "Nose",
    "Throat", "Knee", "Ankle", "Shoulder", "Vertebrae", "Elbow", "Hand", "Foot",
    "Male genitalia", "Female genitalia", "Oral cavity", "Skin", "Head"
]


In [5]:
# define artist function enhanced
def anatomy_generator(anatomy, folder="anatomy_posters", csv_path="anatomy_poster_log.csv"):
    os.makedirs(folder, exist_ok=True)

    anatomy_clean = anatomy.lower().replace(" ", "_")
    today = date.today().isoformat()
    filename = f"popart_{anatomy_clean}_{today}.png"
    filepath = os.path.join(folder, filename)

    prompt = f"Vibrant pop-art style human anatomy of {anatomy}, with anatomical landmarks, labeled parts, and bold retro font text '{anatomy}', suitable for wall art, highly detailed, colorful, 1024x1024 resolution"

    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )

    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    image = Image.open(BytesIO(image_data))
    image.save(filepath)

    log_anatomy_to_csv(anatomy=anatomy, filename=filename, folder=folder, csv_path=csv_path)

    print(f"✅ Image saved: {filepath}")
    return image


In [6]:
for anatomy in medical_anatomy:
    try:
        print(f"🧠 Generating poster for: {anatomy}")
        anatomy_generator(anatomy)
    except Exception as e:
        print(f"❌ Error for {anatomy}: {e}")


🧠 Generating poster for: Heart
📝 CSV updated: anatomy_poster_log.csv
✅ Image saved: anatomy_posters\popart_heart_2025-06-27.png
🧠 Generating poster for: Lung
📝 CSV updated: anatomy_poster_log.csv
✅ Image saved: anatomy_posters\popart_lung_2025-06-27.png
🧠 Generating poster for: Kidney
📝 CSV updated: anatomy_poster_log.csv
✅ Image saved: anatomy_posters\popart_kidney_2025-06-27.png
🧠 Generating poster for: Eye
📝 CSV updated: anatomy_poster_log.csv
✅ Image saved: anatomy_posters\popart_eye_2025-06-27.png
🧠 Generating poster for: Brain
📝 CSV updated: anatomy_poster_log.csv
✅ Image saved: anatomy_posters\popart_brain_2025-06-27.png
🧠 Generating poster for: Bladder
📝 CSV updated: anatomy_poster_log.csv
✅ Image saved: anatomy_posters\popart_bladder_2025-06-27.png
🧠 Generating poster for: Abdomen
📝 CSV updated: anatomy_poster_log.csv
✅ Image saved: anatomy_posters\popart_abdomen_2025-06-27.png
🧠 Generating poster for: Uterus
📝 CSV updated: anatomy_poster_log.csv
✅ Image saved: anatomy_posters

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [ ]:
talker("Well, hi there")


## PC Variation 1

In [ ]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

talker("Well, hi there")

In [ ]:
from pydub import AudioSegment
from io import BytesIO
import tempfile
import os
import simpleaudio as sa

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Also, try replacing onyx with alloy
        input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")

    # Create a temporary file in a folder where you have write permissions
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False, dir=os.path.expanduser("~/Documents")) as temp_audio_file:
        temp_file_name = temp_audio_file.name
        audio.export(temp_file_name, format="wav")
    
    # Load and play audio using simpleaudio
    wave_obj = sa.WaveObject.from_wave_file(temp_file_name)
    play_obj = wave_obj.play()
    play_obj.wait_done()  # Wait for playback to finish

    # Clean up the temporary file afterward
    os.remove(temp_file_name)
    
talker("Well hi there")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a HUGE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>